<a href="https://colab.research.google.com/github/41371112h/114-1/blob/main/HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
from google.auth import default
import pandas as pd
import re

# ====== 1. 授權 ======
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("已完成 Google 帳號授權")

# ====== 2. 開啟試算表 ======
sheet_url = "https://docs.google.com/spreadsheets/d/1HvCj-v296pFicLIjB5BbsX5U26Pd4d76QM5iuF1NeSo/edit?gid=0#gid=0"
sheet_id = re.search(r'/d/([a-zA-Z0-9-_]+)', sheet_url).group(1)

spreadsheet = gc.open_by_key(sheet_id)

# 主表：消費紀錄
try:
    worksheet = spreadsheet.worksheet("消費紀錄")
except gspread.exceptions.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title="消費紀錄", rows=1000, cols=5)
    worksheet.append_row(["日期", "時間", "品項", "分類", "金額"])

# Summary 表
try:
    summary_ws = spreadsheet.worksheet("Summary")
except gspread.exceptions.WorksheetNotFound:
    summary_ws = spreadsheet.add_worksheet(title="Summary", rows=1000, cols=5)

print("已打開工作表: 消費紀錄 + Summary")

# ====== 3. 讀取現有資料到 DataFrame ======
df = get_as_dataframe(worksheet, evaluate_formulas=True, dtype=str)
df = df.dropna(how="all")
if df.empty:
    df = pd.DataFrame(columns=["日期", "時間", "品項", "分類", "金額"])

# ====== 4. 文字輸入並即時寫入 ======
records = []
while True:
    date = input("請輸入日期 (YYYY-MM-DD): ")
    time = input("請輸入時間 (HH:MM): ")
    item = input("請輸入品項: ")
    category = input("請輸入分類 (例如: 餐飲/交通/娛樂): ")
    amount = input("請輸入金額: ")

    record = {"日期": date, "時間": time, "品項": item, "分類": category, "金額": float(amount)}
    records.append(record)
    df = pd.concat([df, pd.DataFrame([record])], ignore_index=True)

    worksheet.append_row([date, time, item, category, float(amount)])
    print("已新增一筆紀錄到 消費紀錄！")

# ====== 5. 更新 Summary ======
    summary = df.groupby("分類")["金額"].sum().reset_index()
    summary.loc[len(summary)] = ["總額", df["金額"].astype(float).sum()]  # 加總額

    summary_ws.clear()
    set_with_dataframe(summary_ws, summary)
    print("Summary 已更新！\n")

# ====== 6. 結束後顯示 ======
print("\n--- 所有紀錄 ---")
print(df)

print("\n--- 分類小計 + 總額 ---")
print(summary)